In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pylab
import pickle
import os

import torchvision
from torchvision import datasets, transforms
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Subset, DataLoader
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from IPython.display import clear_output
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

%matplotlib inline

c:\Users\User\Documents\Hackaton 2022\Attempts\venv\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\User\Documents\Hackaton 2022\Attempts\venv\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class ImageDataset(Dataset):
    def __init__(self, data_folder, description_csv):
        
        self.data_folder = data_folder
        self.description = pd.read_csv(description_csv)

    def __len__(self):
        return self.description.shape[0]

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.to_list()

        images_to_load = list([self.description.iloc[index]['image_name']])
        classes = self.description.iloc[index]['class_id']

        images = []
        for image_relative_path in images_to_load:
            images.append(Image.open(os.path.join(self.data_folder, image_relative_path)))
        return images, classes

In [3]:
class ImageSubset(Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, index):
        images, classes = self.subset[index]

        result = torch.tensor([])
        for image in images:
            result = torch.cat([result, self.transform(image)])

        return result, classes

In [4]:
def train_model(n_epoch, model, criterion, optimizer, train_loader):
    iteration = 0
    loss_history = []
    for epoch in range(n_epoch):
        epoch_loss = []
        for X_batch, y_batch in train_loader:
            model.train()
            
            optimizer.zero_grad()
            logits = model(X_batch.to(device)).cpu()
            loss = criterion(logits, y_batch)
            epoch_loss.append(loss.item())
            
            loss.backward()
            optimizer.step()
            print('+' + '-' * 50)
            print("| Iteration ", iteration + 1, ', loss: ', epoch_loss[-1], sep='')
            print('+' + '-' * 50)
            iteration += 1

        loss_history.append(np.mean(epoch_loss))
        print('/' * 51)
        print("// EPOCH ", epoch + 1, ', loss: ', loss_history[-1], sep='')
        print('/' * 51)

In [5]:
train_transform = torchvision.transforms.Compose([
    transforms.Resize(400),
    transforms.CenterCrop(350),
    transforms.RandomHorizontalFlip(),
    transforms.RandomPerspective(distortion_scale=0.3, p=1),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
test_transform = torchvision.transforms.Compose([
    transforms.Resize(450),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [6]:
DATA_FOLDER = './data/merged/'
MODEL_FOLDER = './models/stacking/'
IMAGE_FOLDER = os.path.join(DATA_FOLDER, 'img/')
DESCRIPTION_PATH = os.path.join(DATA_FOLDER, 'description.csv')
BATCH_SIZE = 20

DESCRIPTION = pd.read_csv(DESCRIPTION_PATH)

DATASET_SIZE = len(DESCRIPTION)
FOLDS_COUNT = 3
FOLD_SIZE = DATASET_SIZE // FOLDS_COUNT + 1
DATASET = ImageDataset(IMAGE_FOLDER, DESCRIPTION_PATH)

In [7]:
false_count = np.unique(DESCRIPTION.values[:, 1], return_counts=True)[1][0]
true_count = np.unique(DESCRIPTION.values[:, 1], return_counts=True)[1][1]

print('With weapon: ', true_count, ', without weapon: ', false_count, sep='')
print('Dataset size:', DATASET_SIZE)

With weapon: 2352, without weapon: 2478
Dataset size: 4830


In [8]:
class ResNet101(nn.Module):
    def __init__(self, pretrained=False):
        super(ResNet101, self).__init__()
        self.model = torchvision.models.resnet101(pretrained=pretrained)
        
        self.model.fc = nn.Linear(2048, 10)
        self.linear = nn.Linear(10, 2)
        
        layers_count = len(list(self.model.parameters()))
        for i, parameter in enumerate(self.model.parameters()):
            if i < layers_count - 5:
                parameter.requires_grad = False
                
                
    def forward(self, X):
        logits = self.model(X)
        logits = self.linear(logits)
        return logits

In [9]:
class ResNet152(nn.Module):
    def __init__(self, pretrained=False):
        super(ResNet152, self).__init__()
        self.model = torchvision.models.resnet152(pretrained=pretrained)
        
        self.model.fc = nn.Linear(2048, 10)
        self.linear = nn.Linear(10, 2)
        
        layers_count = len(list(self.model.parameters()))
        for i, parameter in enumerate(self.model.parameters()):
            if i < layers_count - 10:
                parameter.requires_grad = False
                
                
    def forward(self, X):
        logits = self.model(X)
        logits = self.linear(logits)
        return logits

In [10]:
class DenseNet(nn.Module):
    def __init__(self, pretrained=False):
        super(DenseNet, self).__init__()
        self.model = torchvision.models.densenet201(pretrained=pretrained)
        
        self.model.classifier = nn.Linear(1920, 10)
        self.linear = nn.Linear(10, 2)
        
        layers_count = len(list(self.model.parameters()))
        for i, parameter in enumerate(self.model.parameters()):
            if i < layers_count - 5:
                parameter.requires_grad = False
                
                
    def forward(self, X):
        logits = self.model(X)
        logits = self.linear(logits)
        return logits

In [11]:
resnet101_data = pd.read_csv(os.path.join(MODEL_FOLDER, 'resnet101_data.csv'), index_col=0)
resnet152_data = pd.read_csv(os.path.join(MODEL_FOLDER, 'resnet152_data.csv'), index_col=0)
densenet_data = pd.read_csv(os.path.join(MODEL_FOLDER, 'densenet_data.csv'), index_col=0)

data = pd.concat([resnet101_data, resnet152_data, densenet_data])

In [12]:
X = np.array(data.drop(['y'], axis=1))
y = np.array(data['y'])
X.shape, y.shape

((4830, 10), (4830,))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape

((3864, 10), (966, 10))

In [15]:
forest_parameters = {'criterion': ('gini', 'entropy'), 'n_estimators': range(150, 170)}

forest = RandomForestClassifier()
extra = ExtraTreesClassifier()


In [16]:
forest.fit(X_train, y_train)
extra.fit(X_train, y_train)
print(accuracy_score(forest.predict(X_test), y_test))
print(accuracy_score(extra.predict(X_test), y_test))

0.8612836438923396
0.8581780538302277


In [18]:
pickle.dump(extra, open('extra_tree.pkl', 'wb'))

In [96]:
from catboost import CatBoostClassifier
import catboost

In [92]:
classifier = CatBoostClassifier(iterations=10, learning_rate=0.1)

In [93]:
classifier.fit(X_train, y_train, verbose=True)

0:	learn: 0.6451400	total: 21.1ms	remaining: 190ms
1:	learn: 0.6035284	total: 23.5ms	remaining: 94.2ms
2:	learn: 0.5697062	total: 25.7ms	remaining: 59.9ms
3:	learn: 0.5414449	total: 27.9ms	remaining: 41.8ms
4:	learn: 0.5153028	total: 30.3ms	remaining: 30.3ms
5:	learn: 0.4928609	total: 32.8ms	remaining: 21.8ms
6:	learn: 0.4744028	total: 35ms	remaining: 15ms
7:	learn: 0.4592100	total: 37ms	remaining: 9.26ms
8:	learn: 0.4446717	total: 39.3ms	remaining: 4.37ms
9:	learn: 0.4320416	total: 41.5ms	remaining: 0us


In [94]:
accuracy_score(classifier.predict(X_test), y_test)

0.855072463768116

In [95]:
classifier.predict_proba(X_test)

array([[0.60786299, 0.39213701],
       [0.76991236, 0.23008764],
       [0.23895998, 0.76104002],
       ...,
       [0.24075949, 0.75924051],
       [0.67597783, 0.32402217],
       [0.2504208 , 0.7495792 ]])

In [97]:
classifier.save_model('boosting.model')